In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as pltimport

train=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")
print(train.shape)
print(test.shape)

combined=pd.concat([train,test])
combined.shape #2919
#combined.iloc[:1460:,:] #train
#combined.iloc[1460:,:] #test

combined=combined.reset_index()

from preprocess import impute
all_data, encodedDic = impute (combined, False)
all_data_onehot, encodedDic = impute (combined, True) #one-hot

train=all_data.iloc[:1460:,:] #train
train_y=train["SalePrice"]
train_x=train[train.columns[train.columns!="SalePrice"]]
test=test.iloc[1460:,:]

train_onehot=all_data_onehot.iloc[:1460:,:] #train
train_y_onehot=train_onehot["SalePrice"]
train_x_onehot=train_onehot[train_onehot.columns[train_onehot.columns!="SalePrice"]]
test_onehot=train_onehot.iloc[1460:,:]


In [ ]:
train_onehot[forbx]

In [ ]:
import matplotlib.gridspec as gridspec
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

gs=gridspec.GridSpec(251,1)
plt.figure(figsize=(16, 340))
for i in range(train_onehot.shape[1]):
    ax1 = plt.subplot(gs[i, 0])
    ax1.set_title('')
    ax1.set_ylabel("")
    ax1.set_xlabel("")
#     xlim=ax1.get_xlim()  
    sns.distplot(train_onehot.iloc[:,i])
    for tick in ax1.xaxis.get_major_ticks():
        tick.label1On = False
        tick.label2On = True
#     for tick in ax1.yaxis.get_major_ticks():
#         tick.label1On=True

    
plt.show()

In [ ]:
# subset numerical variable that look skewed
forbx=["1stFlrSF","2ndFlrSF","GarageArea","GrLivArea","LotArea","LotFrontage","SalePrice","TotalBsmtSF","TotalFlrSF","Bsmt_Unf","TotalProchSF"]

In [ ]:
# check the skewness of the selected column
from scipy.stats import skew
skewed_data=all_data_onehot[forbx].apply(lambda x: skew(x))
skewed = skewed_data[abs(skewed_data) > 0.75]
skewed

In [ ]:
from scipy.special import boxcox1p
import matplotlib.pyplot as plt

para=np.linspace(0.0005,0.00001, 50) #lambda tuning

tunning_result=boxcox1p(skewed[5],para)
xlen=len(tunning_result)
df_= pd.DataFrame(tunning_result, index=range(xlen))


# title = 'Ridge mse as a function of the regularization'
axes = df_.plot()
axes.set_xlabel('lambda')
axes.set_ylabel('skewness')
plt.show()

In [ ]:
for i in skewed.index:
    all_data_onehot[i]=boxcox1p(all_data_onehot[i],0.001)
    print(train_onehot[i])

In [ ]:
sns.distplot(train_onehot["SalePrice"])

In [ ]:
all_data_onehot.to_csv("bx_all_data_onehot.csv",index=False)

In [ ]:
all_data_=pd.read_csv("bx_all_data_onehot.csv")


bx_train_onehot=all_data_.iloc[:1460:,:] #train
bx_train_y_onehot=bx_train_onehot["SalePrice"]
bx_train_x_onehot=bx_train_onehot[bx_train_onehot.columns[bx_train_onehot.columns!="SalePrice"]]
bx_test_onehot=bx_train_onehot.iloc[1460:,:]


### Ridge Regression Comparison with BoxCox

In [ ]:
## With BX
from sklearn.model_selection import GridSearchCV # search for the best lambda
from sklearn import linear_model

ridge = linear_model.Ridge(normalize=True) # create a ridge regression instance

# find the best alpha (lambda) for ridge
grid_param = [{'alpha': np.logspace(-4, 2, 100)}]
para_search_ridge = GridSearchCV(estimator=ridge, param_grid=grid_param, scoring='neg_mean_squared_error', cv=5, return_train_score=True)
para_search_ridge.fit(bx_train_x_onehot, bx_train_y_onehot)

print(para_search_ridge.best_params_)
print("Lowest RMSE found: ", np.sqrt(np.abs(para_search_ridge.best_score_)))

# fit best ridge equation to all train data
best_ridge_y = para_search_ridge.best_estimator_.predict(bx_train_x_onehot)
print("RMSE: ", np.sqrt(np.mean((train_y_onehot-best_ridge_y)**2)))

In [ ]:
# No BX
from sklearn.model_selection import GridSearchCV # search for the best lambda
from sklearn import linear_model

ridge = linear_model.Ridge(normalize=True) # create a ridge regression instance

# find the best alpha (lambda) for ridge
grid_param = [{'alpha': np.logspace(-4, 2, 100)}]
para_search_ridge = GridSearchCV(estimator=ridge, param_grid=grid_param, scoring='neg_mean_squared_error', cv=5, return_train_score=True)
para_search_ridge.fit(train_x_onehot, train_y_onehot)

print(para_search_ridge.best_params_)
print("Lowest RMSE found: ", np.sqrt(np.abs(para_search_ridge.best_score_)))

# fit best ridge equation to all train data
best_ridge_y = para_search_ridge.best_estimator_.predict(train_x_onehot)
print("RMSE: ", np.sqrt(np.mean((train_y_onehot-best_ridge_y)**2)))

### Lasso Regression Comparion with BoxCox

In [ ]:
## NO BX
from sklearn.model_selection import GridSearchCV # search for the best lambda
from sklearn import linear_model

lasso= linear_model.Lasso(normalize=True) # create a ridge regression instance

# find the best alpha (lambda) for ridge
grid_param = [{'alpha': np.logspace(-4.5, 2, 100)}]
para_search_lasso = GridSearchCV(estimator=lasso, param_grid=grid_param, scoring='neg_mean_squared_error', cv=5, return_train_score=True)
para_search_lasso.fit(train_x_onehot, train_y_onehot)

print(para_search_lasso.best_params_)
print("Lowest RMSE found: ", np.sqrt(np.abs(para_search_lasso.best_score_)))

# fit best ridge equation to all train data
best_lasso_y = para_search_lasso.best_estimator_.predict(train_x_onehot)
print("RMSE: ", np.sqrt(np.mean((train_y_onehot-best_lasso_y)**2)))

# {'alpha': 25.65020905680051}
# Lowest RMSE found:  33809.958651596615
# RMSE:  27539.866355369504

In [ ]:
## BX
from sklearn.model_selection import GridSearchCV # search for the best lambda
from sklearn import linear_model

lasso= linear_model.Lasso(normalize=True) # create a ridge regression instance

# find the best alpha (lambda) for ridge
grid_param = [{'alpha': np.logspace(-4.5, 2, 100)}]
para_search_lasso = GridSearchCV(estimator=lasso, param_grid=grid_param, scoring='neg_mean_squared_error', cv=5, return_train_score=True)
para_search_lasso.fit(bx_train_x_onehot, bx_train_y_onehot)

print(para_search_lasso.best_params_)
print("Lowest RMSE found: ", np.sqrt(np.abs(para_search_lasso.best_score_)))

# fit best ridge equation to all train data
best_lasso_y = para_search_lasso.best_estimator_.predict(bx_train_x_onehot)
print("RMSE: ", np.sqrt(np.mean((bx_train_y_onehot-best_lasso_y)**2)))

# {'alpha': 25.65020905680051}
# Lowest RMSE found:  32559.718946794015
# RMSE:  28119.588145632715

## Elastic Net Comparison with BoxCox

In [ ]:
## with BX
from sklearn.model_selection import GridSearchCV # search for the best lambda
from sklearn import linear_model

EN= linear_model.ElasticNet(normalize=True) # create a ridge regression instance

# find the best alpha (lambda) for ridge
grid_param = [{'alpha': np.logspace(-4.5, 2, 50),'l1_ratio':np.linspace(0.1,0.9,20)}]
para_search_EN = GridSearchCV(estimator=EN, param_grid=grid_param, scoring='neg_mean_squared_error', cv=5, return_train_score=True)
para_search_EN.fit(bx_train_x_onehot, bx_train_y_onehot)

print(para_search_EN.best_params_)
print("Lowest RMSE found: ", np.sqrt(np.abs(para_search_EN.best_score_)))

# fit best ridge equation to all train data
best_EN_y = para_search_EN.best_estimator_.predict(bx_train_x_onehot)
print("RMSE: ", np.sqrt(np.mean((bx_train_y_onehot-best_EN_y)**2)))

# {'alpha': 0.0004941713361323838, 'l1_ratio': 0.3526315789473684}
# Lowest RMSE found:  32670.8665945391
# RMSE:  27932.17975891021

In [ ]:
from myfunction import run_GLM

In [ ]:
from myfunction import run_GB

In [ ]:
run_GB(bx_train_x_onehot,bx_train_y_onehot)